In [1]:
import math

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from tqdm import tqdm

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
class WineModel(nn.Module):
    def __init__(self, device, feature_count, hidden_count, output_count):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(feature_count, hidden_count, device=device),
            nn.ReLU(),
            nn.Linear(hidden_count, hidden_count, device=device),
            nn.ReLU(),
            nn.Linear(hidden_count, output_count, device=device),
            nn.Softmax()
        )
    
    def forward(self, features, targets):
        b, t = features.size()
        logits = self.model(features)
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss

In [4]:
class WineDataset(Dataset):
    def __init__(self, df):
        self.train = df.drop(['quality'], axis=1)
        self.target = df['quality']
        print(self.target.shape, self.train.shape)
        
    def __len__(self):
        return len(self.train)
    
    def __getitem__(self, index):
        # make an onehot vector
        target = self.target.iloc[index]
        result = [0] * 11
        result[target] = 1
        return torch.tensor(data=self.train.iloc[index, :].to_numpy(), dtype=torch.float), torch.tensor(data=result, dtype=torch.float)

In [5]:
train_read = pd.read_csv('data/concat_processed.csv')
feature_count = len(train_read.columns) - 1  # minus 1 for target

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(train_read, test_size=0.1)

In [6]:
from torch.utils.data import DataLoader

train_dataset = WineDataset(train_df)
test_dataset = WineDataset(test_df)

(4352,) (4352, 12)
(2145,) (2145, 12)


In [7]:
class TrainerConfig:
    lr = 4e-4
    batch_size = 256
    num_workers = 6
    max_epochs = 1000
    
trainer_config = TrainerConfig()

In [8]:
class Trainer:
    def __init__(self, model, train_dataset, test_dataset, config, device):
        self.config = config
        self.device = device
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        
    def train(self):
        config = self.config
        model = self.model
        optimizer = optim.AdamW(self.model.parameters(), lr=config.lr)
        
        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset
            loader = DataLoader(data, batch_size=config.batch_size, num_workers=config.num_workers)

            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            for it, (x, y) in pbar:

                # place data on the correct device
                x = x.to(self.device)
                y = y.to(self.device)

                # forward the model
                with torch.set_grad_enabled(is_train):
                    logits, loss = model(x, y)
                    loss = loss.mean() # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())

                if is_train:

                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    # torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()

                    # report progress
                    # if it % 10 == 0:
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {self.config.lr:e}")

            if not is_train:
                logger.info("test loss: %f", np.mean(losses))

            return losses
        
        losses = []
        for epoch in range(config.max_epochs):
            losses.extend(run_epoch('train'))
            if self.test_dataset is not None:
                run_epoch('test')
        plt.plot(losses)
        plt.xlabel('#iterations')
        plt.ylabel('training loss')
        plt.savefig('train.png')


In [9]:
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
model = WineModel(device, feature_count, 4, 11)
trainer = Trainer(model, train_dataset, None, trainer_config, device)

In [10]:
trainer.train()

  0%|          | 0/68 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
epoch 1 iter 67: train loss 2.23306. lr 8.000000e-03: 100%|██████████| 68/68 [00:00<00:00, 178.44it/s]
epoch 2 iter 67: train loss 2.10022. lr 8.000000e-03: 100%|██████████| 68/68 [00:00<00:00, 283.84it/s]
epoch 3 iter 67: train loss 2.10168. lr 8.000000e-03: 100%|██████████| 68/68 [00:00<00:00, 286.23it/s]
epoch 4 iter 67: train loss 2.10177. lr 8.000000e-03: 100%|██████████| 68/68 [00:00<00:00, 272.90it/s]
epoch 5 iter 1: train loss 2.12046. lr 8.000000e-03:   3%|▎         | 2/68 [00:00<00:00, 132.31it/s]


KeyboardInterrupt: 

In [ ]:
# evaluate
correct_count = 0
predict, _ = model(
    torch.tensor(data=test_df.drop('quality', axis=1).to_numpy(), dtype=torch.float).to(device),
    torch.tensor(data=test_df['quality'].to_numpy(), dtype=torch.long).to(device)
)
predict.shape

/home/anarion/anaconda3/envs/local_nmt/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


torch.Size([528, 11])

In [ ]:
print(torch.sum(torch.eq(torch.argmax(predict, dim=1), torch.tensor(test_df['quality'].to_numpy()).to(device))) / len(test_df))

tensor(0.5871)